In [ ]:
import pandas as pd
import numpy as np
import sqlite3 as db
from datetime import datetime
import urllib
import os
import requests
from pyquery import PyQuery  


con = db.connect(database="../db.sqlite")

In [ ]:
cursor = con.cursor()
cursor.execute("select CountryCode, Name from Country")

list = []

for row in cursor.fetchall():
    list.append({
        "CountryCode": row[0],
        "Country": row[1]
    })
    

def getCountryCode(name):
    
    if name == "United Kingdom":
        return "GBR"
    elif name == "Russian Federation":
        return "RUS"
    elif name == "Korea":
        return "KOR"
    elif name == "Slovak Republic":
        return "SVK"
    
    for c in list:
        if c["Country"] == name:
            return c["CountryCode"]
        
    return None

In [ ]:
cursor = con.cursor()
cursor.execute("select CurrencyCode from Currency")

currencyList = []

for row in cursor.fetchall():
    currencyList.append(row[0])

In [ ]:
def crawlCurrency(code):
    link = "https://finance.yahoo.com/quote/"+code+"=X/history"
    print(link)
    
    
    result = requests.get(link, headers={
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0",
    })
    
    pq = PyQuery(result.text)
    secondRow =  PyQuery(pq('table[data-test="historical-prices"] tr:nth-child(2)'))
    
    dateTxt = secondRow("td:nth-child(1)").text()
    close = secondRow("td:nth-child(5)").text()
    
    date = None
    try:
        date = datetime.strptime(dateTxt, "%b %d, %Y")
    except:
        return
        
    
    
    
      
    cursor.execute("""select count(*) 
                from CurrencyUSDValue 
                where Currency_id = ? 
                    and Date = ? """, (currencyCode,date,))
    
    
    
    count = cursor.fetchall()[0][0]
    
    if count == 0:
        print("inserting: " + currencyCode + " " + date.strftime("%d %m %Y"))
        cursor.execute("""insert into CurrencyUSDValue(Currency_id, Date, Close) Values(?,?,?)""",
                       (currencyCode,date,close,))
        print("inserted")
    else:
        print(currencyCode +  " " + date.strftime("%d %m %Y") +" already exists")
    
    
    

In [ ]:
for currencyCode in currencyList:
    crawlCurrency(currencyCode)
con.commit()